In [94]:
%load_ext autoreload
%autoreload 2

import scod
import torch
import torch.nn as nn
import numpy as np
from tqdm import trange
import datetime
import random
import pickle


from torchvision import transforms
from torchvision.datasets import MNIST, FashionMNIST, KMNIST, EMNIST
from torch.utils.data import Subset
from torch.utils.data import DataLoader

import seaborn as sns
from matplotlib import pyplot as plt

from scripts.train_functions import create_dataloaders, main, set_up_model, load_ckp
from scripts.benchmark_functions import alg_flags, scod_flagger, ds_scod_flagger, create_scod_model, create_benchmark_seq_batches, get_Lt_J
from scripts.utils import eval_scod, init_scod


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
root="~/data/"
batch_size = 100
dataset_name = "mnist"


In [3]:
dataloaders, dataset_sizes = create_dataloaders(root, batch_size, dataset_name=dataset_name)

print(dataloaders)

{'mnist_train': <torch.utils.data.dataloader.DataLoader object at 0x7fb0ab513e90>, 'mnist_test': <torch.utils.data.dataloader.DataLoader object at 0x7fb0ab513cd0>, 'fashion': <torch.utils.data.dataloader.DataLoader object at 0x7fb0ab513fd0>, 'kmnist': <torch.utils.data.dataloader.DataLoader object at 0x7fb07433d050>, 'emnist': <torch.utils.data.dataloader.DataLoader object at 0x7fb07433d1d0>}


# Add SCOD wrapper

In [4]:
load_model_path = "./best_model/mnist/best_model.pt"

unc_model = create_scod_model(load_model_path, dataset_name, batch_size)

Weight space dimension: 4.443e+04
computing basis
using T = 304


  0%|          | 0/480 [00:00<?, ?it/s]

# Create mixture dataset using flagged inputs

In [157]:
ood_ratio = 0.99
mix_train_ratio = 0.01
num_batches = 10

ood_num = int(batch_size*ood_ratio)
mix_train_num = int(batch_size*mix_train_ratio)

batch_compositions = [  {'mnist_test':batch_size - ood_num,'fashion':ood_num}]*num_batches
train_batch_compositions = [{'mnist_train':mix_train_num}]*num_batches

flag_limit = ood_num
# algs_to_test = [lambda x: scod_flagger(x, unc_model, flag_limit=flag_limit, debug = False, dist_layer = scod.distributions.CategoricalLogitLayer()), 
#                 lambda x: ds_scod_flagger(x, unc_model, flag_limit=flag_limit, dist_layer = scod.distributions.CategoricalLogitLayer())]
algs_to_test = [lambda x: scod_flagger(x, unc_model, flag_limit=flag_limit, debug = False, dist_layer = scod.distributions.CategoricalLogitLayer())]
algs_names = ['scod_flagged']

flagged_inputs, flagged_labels, flagged_fnames, flags = alg_flags(algs_to_test, dataset_name, batch_size, num_batches, batch_compositions)
train_inputs, train_labels, train_fnames = create_benchmark_seq_batches(dataset_name, mix_train_num, num_batches, train_batch_compositions)
for k in range(len(algs_to_test)):
    for i in range(num_batches):
        flagged_subset = [(ts, lbl) for (ts,lbl,flg) in zip(flagged_inputs[i], flagged_labels[i], flags[i][k]) if flg]
        train_subset = [(ts, lbl) for (ts,lbl) in zip(train_inputs[i], train_labels[i])]
        refine_subset = flagged_subset + train_subset
        random.shuffle(refine_subset)

        mix_name = "mixture_"+str(batch_compositions[i])+'_'+algs_names[k]+'_'+str(len(flagged_subset))+'_train_'+str(len(train_subset))+"_batch_"+str(i)
        
        dataloaders[mix_name] = DataLoader(refine_subset, batch_size=batch_size, shuffle=True, num_workers=8)
        dataset_sizes[mix_name]  = len(refine_subset)
        print("Added ", mix_name, " to list of dataloaders.")

Test seq  0  algorithm  0  flagged:  [False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]
Test seq  1  algorithm  0  flagged:  [False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True

# Resume training

start_epoch_idx =  17
valid_loss = 0.042841


In [162]:


# Resume training on train set
# phases = [('train','mnist_train'), ('val','mnist_test'), ('val','fashion'), ('val','kmnist')]

# Resume training on mixture
# phases = [('train',"mixture_{'mnist_test': 90, 'fashion': 10}_scod_flagged_10_train_100"), 
#             ('val','mnist_test'), ('val','fashion'), ('val','kmnist')]
# phases = [('train',"mixture_{'mnist_test': 90, 'fashion': 10}_scod_flagged_10_train_1"), 
#             ('val','mnist_test'), ('val','fashion'), ('val','kmnist')]
# phases = [('train',"mixture_{'mnist_test': 1, 'fashion': 99}_scod_flagged_99_train_1"), 
#             ('val','mnist_test'), ('val','fashion'), ('val','kmnist')]

for b in range(num_batches):
    initialized_model, exp_lr_scheduler, sgd_optimizer, criterion, device = set_up_model(dataset_name)
    load_model_path = "./best_model/mnist/best_model.pt"
    model, optimizer, start_epoch_idx, valid_loss = load_ckp(load_model_path, initialized_model, sgd_optimizer)
    print("start_epoch_idx = ", start_epoch_idx)
    print("valid_loss = {:.6f}".format(valid_loss))

    start_epochs = start_epoch_idx
    num_epochs = 15
    scheduler = exp_lr_scheduler

    phases = [('train',"mixture_{'mnist_test': 1, 'fashion': 99}_scod_flagged_99_train_1_batch_"+str(b)), 
            ('val','mnist_test'), ('val','fashion'), ('val','kmnist')]

    losses = {p[1]: np.zeros(num_epochs) for p in phases}

    # Resume training
    for epoch in range(start_epochs, start_epochs+num_epochs):
        print('Epoch {}/{}'.format(epoch+1, start_epochs+num_epochs))
        print('-' * 10)
        for phase in phases:
            if phase[0] == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()

            running_loss = 0.0

            # batch loop
            for d in dataloaders[phase[1]]:
                if len(d) == 2:
                    inputs, labels = d 
                elif len(d) == 3:
                    inputs, labels, fnames = d 
                else:
                    raise ValueError("Unrecognized. Dataloader entry has length {}".format(len(d)))
                labels = labels.flatten().type(torch.LongTensor)

                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase[0] == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    if phase[0] == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
            epoch_loss = running_loss / dataset_sizes[phase[1]]
            print('{} {} Loss: {:.4f}'.format(phase[0], phase[1], epoch_loss))
            if phase[0] == 'train':
                scheduler.step()
            losses[phase[1]][epoch-start_epochs] = epoch_loss

    # Save train losses and valid losses
    losses_path = "./losses/"
    fname = "batch_"+str(b)+"_"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S") +'.npz' 
    with open(losses_path+fname, "wb") as fp:
        pickle.dump(losses, fp)



Epoch 18/32
----------
train mixture_{'mnist_test': 1, 'fashion': 99}_scod_flagged_99_train_1_batch_0 Loss: 5.7131
val mnist_test Loss: 0.0428
val fashion Loss: 5.9405
val kmnist Loss: 8.5930
Epoch 19/32
----------
train mixture_{'mnist_test': 1, 'fashion': 99}_scod_flagged_99_train_1_batch_0 Loss: 5.6765
val mnist_test Loss: 0.0427
val fashion Loss: 5.8659
val kmnist Loss: 8.5580
Epoch 20/32
----------
train mixture_{'mnist_test': 1, 'fashion': 99}_scod_flagged_99_train_1_batch_0 Loss: 5.6083
val mnist_test Loss: 0.0426
val fashion Loss: 5.7625
val kmnist Loss: 8.5091
Epoch 21/32
----------
train mixture_{'mnist_test': 1, 'fashion': 99}_scod_flagged_99_train_1_batch_0 Loss: 5.5136
val mnist_test Loss: 0.0425
val fashion Loss: 5.6360
val kmnist Loss: 8.4483
Epoch 22/32
----------
train mixture_{'mnist_test': 1, 'fashion': 99}_scod_flagged_99_train_1_batch_0 Loss: 5.3973
val mnist_test Loss: 0.0425
val fashion Loss: 5.4923
val kmnist Loss: 8.3779
Epoch 23/32
----------
train mixture_{'m

KeyboardInterrupt: 

In [129]:
# Save train losses and valid losses
losses_path = "./losses/"
fname = datetime.datetime.now().strftime("%Y%m%d-%H%M%S") +'.npz' 
# with open(losses_path+fname, "wb") as fp:
#     pickle.dump(losses, fp)

# Resume training in a new way

In [91]:
initialized_model, exp_lr_scheduler, sgd_optimizer, criterion, device = set_up_model(dataset_name)
load_model_path = "./best_model/mnist/best_model.pt"
model, optimizer, start_epoch_idx, valid_loss = load_ckp(load_model_path, initialized_model, sgd_optimizer)
print("start_epoch_idx = ", start_epoch_idx)
print("valid_loss = {:.6f}".format(valid_loss))

start_epoch_idx =  17
valid_loss = 0.042841


In [98]:
start_epochs = start_epoch_idx
num_epochs = 1
scheduler = exp_lr_scheduler

# Resume training on train set
# phases = [('train','mnist_train'), ('val','mnist_test'), ('val','fashion'), ('val','kmnist')]

# Resume training on mixture
phases = [('train',"mixture_{'mnist_test': 90, 'fashion': 10}_scod_flagged_10_train_100"), 
            ('val','mnist_test'), ('val','fashion'), ('val','kmnist')]

losses = {p[1]: np.zeros(num_epochs) for p in phases}

dist_layer = scod.distributions.CategoricalLogitLayer()

# Resume training
for epoch in range(start_epochs, start_epochs+num_epochs):
    print('Epoch {}/{}'.format(epoch+1, start_epochs+num_epochs))
    print('-' * 10)

    # Create new unc model
    new_unc_model = init_scod(model, dataloaders["mnist_train"], dataset_name) 

    for phase in phases:
        if phase[0] == 'train':
            model.train()  # Set model to training mode
        else:
            model.eval()

        running_loss = 0.0

        # batch loop
        for d in dataloaders[phase[1]]:
            if len(d) == 2:
                inputs, labels = d 
            elif len(d) == 3:
                inputs, labels, fnames = d 
            else:
                raise ValueError("Unrecognized. Dataloader entry has length {}".format(len(d)))
            labels = labels.flatten().type(torch.LongTensor)

            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase[0] == 'train'):
                outputs = model(inputs)
                loss = criterion(outputs, labels)

                

                if phase[0] == 'train':
                    for i in inputs[:2]:
                        unc = eval_scod(i, unc_model, dist_layer)
                        unc = unc.item()
                        print("Uncertainty: ", unc)
                        LtJ = get_Lt_J(i.unsqueeze(0), unc_model, debug=False, dist_layer=dist_layer)
                        orig_trace = np.trace(LtJ)
                        print("LtJ shape is ", np.shape(LtJ))

                        new_unc = eval_scod(i, new_unc_model, dist_layer)
                        new_unc = new_unc.item()
                        print("New Uncertainty: ", new_unc)
                        new_LtJ = get_Lt_J(i.unsqueeze(0), new_unc_model, debug=False, dist_layer=dist_layer)
                        diff = np.trace(new_LtJ - LtJ)
                        print("Trace is ", diff/orig_trace)


                    loss.backward()
                    optimizer.step()
            running_loss += loss.item() * inputs.size(0)
        epoch_loss = running_loss / dataset_sizes[phase[1]]
        print('{} {} Loss: {:.4f}'.format(phase[0], phase[1], epoch_loss))
        if phase[0] == 'train':
            scheduler.step()
        losses[phase[1]][epoch-start_epochs] = epoch_loss

print(losses)

Epoch 18/18
----------
Weight space dimension: 4.443e+04
computing basis
using T = 304


  0%|          | 0/480 [00:00<?, ?it/s]

Uncertainty:  2.294168472290039
LtJ shape is  (10, 44426)
New Uncertainty:  2.294233798980713
Trace is  0.0057763914
Uncertainty:  2.2781007289886475
LtJ shape is  (10, 44426)
New Uncertainty:  2.2781505584716797
Trace is  0.039836433
Uncertainty:  2.2669413089752197
LtJ shape is  (10, 44426)
New Uncertainty:  2.266752243041992
Trace is  -0.010459036
Uncertainty:  2.258082151412964
LtJ shape is  (10, 44426)
New Uncertainty:  2.2576828002929688
Trace is  0.052654635
train mixture_{'mnist_test': 90, 'fashion': 10}_scod_flagged_10_train_100 Loss: 0.3520
val mnist_test Loss: 0.0427
val fashion Loss: 5.8917
val kmnist Loss: 8.5776
{"mixture_{'mnist_test': 90, 'fashion': 10}_scod_flagged_10_train_100": array([0.35199595]), 'mnist_test': array([0.04272059]), 'fashion': array([5.89165637]), 'kmnist': array([8.57758451])}


# Create mixture dataset using flagged inputs and weighting by uncertainty

In [84]:
ood_ratio = 0.1
mix_train_ratio = 1.0
num_batches = 1

ood_num = int(batch_size*ood_ratio)
mix_train_num = int(batch_size*mix_train_ratio)

batch_compositions = [  {'mnist_test':batch_size - ood_num,'fashion':ood_num}]
train_batch_compositions = [{'mnist_train':mix_train_num}]

flag_limit = ood_num
# algs_to_test = [lambda x: scod_flagger(x, unc_model, flag_limit=flag_limit, debug = False, dist_layer = scod.distributions.CategoricalLogitLayer()), 
#                 lambda x: ds_scod_flagger(x, unc_model, flag_limit=flag_limit, dist_layer = scod.distributions.CategoricalLogitLayer())]
algs_to_test = [lambda x: scod_flagger(x, unc_model, flag_limit=flag_limit, debug = False, dist_layer = scod.distributions.CategoricalLogitLayer())] 
algs_names = ['scod_flagged']

flagged_inputs, flagged_labels, flagged_fnames, flags = alg_flags(algs_to_test, dataset_name, batch_size, num_batches, batch_compositions)
train_inputs, train_labels, train_fnames = create_benchmark_seq_batches(dataset_name, mix_train_num, num_batches, train_batch_compositions)
for k in range(len(algs_to_test)):
    for i in range(num_batches):
        flagged_subset = [(ts, lbl) for (ts,lbl,flg) in zip(flagged_inputs[i], flagged_labels[i], flags[i][k]) if flg]
        train_subset = [(ts, lbl) for (ts,lbl) in zip(train_inputs[i], train_labels[i])]
        
        if unc_model is not None:
            dist_layer = scod.distributions.CategoricalLogitLayer()
            for fs in flagged_subset:
                unc = eval_scod(fs[0] , unc_model, dist_layer)
                unc = unc.item()
                print("Uncertainty: ", unc)
        
        # refine_subset = flagged_subset + train_subset
        # random.shuffle(refine_subset)

        # mix_name = "mixture_weighted"+str(batch_compositions[i])+'_'+algs_names[k]+'_'+str(len(flagged_subset))+'_train_'+str(len(train_subset))
        
        # dataloaders[mix_name] = DataLoader(refine_subset, batch_size=batch_size, shuffle=True, num_workers=8)
        # dataset_sizes[mix_name]  = len(refine_subset)
        # print("Added ", mix_name, " to list of dataloaders.")

Test seq  0  algorithm  0  flagged:  [False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, True, True, True, False, True, True, True]
Uncertainty:  2.293386936187744
Uncertainty:  2.296449899673462
Uncertainty:  2.2916970252990723
Uncertainty:  2.295175552368164
Uncertainty:  2.2911081314086914
Uncertainty:  2.2927355766296387
Uncertainty:  2.294149398803711
Uncertainty:  2.2983007431030273
Uncertainty: